In [1]:
%reload_ext autoreload
%autoreload 2

import plunk.sb.sig_compatibility.sig_comp as sc

In [19]:
def foo(a, /,b, *args, c,  d=10,**kwargs):
    pass

In [5]:
d= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)


In [6]:
d

DefinitionSig(PO=1, PK=1, VP=0, KO=2, VK=1)

In [7]:
c = sc.CallSig(pos=1, k=2) # foo(a, b=2, c=3)

In [8]:
d= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)

e= sc.DefinitionSig(PO= 1, PK= 1, VP= 0, KO= 2, VK=1)

cc = sc.CallSig(pos=2, k=1)

In [9]:
sc.call_is_compatible(d,cc)

False

In [10]:
sc.is_compatible_with(d,e)

True

In [11]:
from i2 import Sig
sig = Sig(foo)

In [12]:
for att in dir(sig):
    if att.startswith('_') or callable(getattr(sig, att)):
        continue
    print(att, getattr(sig,att))

KEYWORD_ONLY KEYWORD_ONLY
POSITIONAL_ONLY POSITIONAL_ONLY
POSITIONAL_OR_KEYWORD POSITIONAL_OR_KEYWORD
VAR_KEYWORD VAR_KEYWORD
VAR_POSITIONAL VAR_POSITIONAL
annotations {}
defaults {'d': 10}
has_var_keyword True
has_var_kinds True
has_var_positional True
index_of_var_keyword 5
index_of_var_positional 2
kinds {'a': <_ParameterKind.POSITIONAL_ONLY: 0>, 'b': <_ParameterKind.POSITIONAL_OR_KEYWORD: 1>, 'args': <_ParameterKind.VAR_POSITIONAL: 2>, 'c': <_ParameterKind.KEYWORD_ONLY: 3>, 'd': <_ParameterKind.KEYWORD_ONLY: 3>, 'kwargs': <_ParameterKind.VAR_KEYWORD: 4>}
n_required 3
name foo
names ['a', 'b', 'args', 'c', 'd', 'kwargs']
parameters OrderedDict([('a', <Parameter "a">), ('b', <Parameter "b">), ('args', <Parameter "*args">), ('c', <Parameter "c">), ('d', <Parameter "d=10">), ('kwargs', <Parameter "**kwargs">)])
params [<Parameter "a">, <Parameter "b">, <Parameter "*args">, <Parameter "c">, <Parameter "d=10">, <Parameter "**kwargs">]
var_keyword_name kwargs
var_positional_name args


In [17]:
from i2.signatures import PO, PK, VP, KO, VK
from plunk.sb.sig_compatibility.sig_comp import kind_to_symbol_func, transform_key,var_names_by_kind,kind_to_symbol, possible_named_args




ImportError: cannot import name 'possible_named_args' from 'plunk.sb.sig_compatibility.sig_comp' (/Users/sylvain/Desktop/dev/otosense/plunk/plunk/sb/sig_compatibility/sig_comp.py)

In [14]:
var_names_by_kind(sig)

{'PO': ['a'], 'PK': ['b'], 'VP': ['args'], 'KO': ['c', 'd'], 'VK': ['kwargs']}

In [15]:
def possible_named_args(sig):
    d = var_names_by_kind(sig)
    return d['PK']+d['KO']


In [16]:
possible_named_args(sig)

['b', 'c', 'd']

# sig from params

In [3]:
# (po1, po2, /, pk1, pk2, *, ko1, ko2)
from i2.signatures import PO, PK, KO, VK, VP
params = [
                dict(name='po1', kind=PO),
                dict(name='po2', kind=PO),
                dict(name='pk1', kind=PK),
                dict(name='pk2', kind=PK),
                dict(name='ko1', kind=KO),
                dict(name='ko2', kind=KO),
            ]

In [52]:
import random

def to_zero_one(n):
    return 1 if (n!=0) else 0

def tuple_to_params(param_lengths):
    assert len(param_lengths)==5
    po, pk, vp, ko, vk = param_lengths
    vp = to_zero_one(vp)
    vk = to_zero_one(vk)

def raises_error(func, *args, **kwargs)->bool:
    try:
        func(*args, **kwargs)
        return True
    except TypeError:
        return False

def generate_input_po(po_names):
    po = len(po_names)
    num = random.randint(0, po*2)
    return tuple(random.randint(0,100) for _ in range(num))

def generate_input_pk(pk_names):
    pk = len(pk_names)
    #random.shuffle(pk_names)
    split = random.randint(0,pk)
    print(split)
    first = generate_input_po(pk_names[:split])
    pool_names = [f'c_{i}' for i in range(2*(pk-split))]+pk_names[split:]
    
    second_vals =  generate_input_po(pk_names[split:])
    n_vals = len(second_vals)

    second  = {k:v for k,v in zip(pk_names[split:], second_vals) }
    return first, second



func1 <= func2 = wrapper(func1)
# define "two signatures are compatible" 
# too strong: params and return annots are equal (same length, names, kinds)
# allowed calls included in allowed calls for 2


In [53]:
generate_input_pk(['a','b','c','d','e'])

0


((), {'a': 41, 'b': 47, 'c': 31, 'd': 47, 'e': 5})

# options: generate function from specs like (po, pk, vp, ko, vk)
# sig to func
(po, pk, vp, ko, vk) -> 

In [37]:
random.shuffle(['a','b','c','d','e'])

In [42]:
def f(a,b,c,d,e):
    return a,b,c,d,e

In [45]:
f(1,2,3,4, c=2)

TypeError: f() got multiple values for argument 'c'

In [22]:
sig.bind_partial(1,2,3)

<BoundArguments (a=1, b=2, args=(3,))>

In [12]:
def myf(a00, a01, /, a12, a13, a14, a36):
    return (a00, a01, a12, a13, a14, a36)

In [10]:
from i2.tests.util import sig_to_inputs

In [11]:
list(sig_to_inputs(myf))

[((0, 1), {'a12': 2, 'a13': 3, 'a14': 4, 'a36': 5}),
 ((0, 1, 2), {'a13': 3, 'a14': 4, 'a36': 5}),
 ((0, 1, 2, 3), {'a14': 4, 'a36': 5}),
 ((0, 1, 2, 3, 4), {'a36': 5}),
 ((0, 1, 2, 3, 4, 5), {})]

In [52]:
from collections import Counter
from i2.tests.util import sig_to_inputs
from i2.signatures import VP, VK, PO, PK, KO

def sig_to_func(sig):  # check Thor's function: could not find it
    @Sig(sig)
    def func(*args, **kwargs):
        return args, kwargs

    return func


def variadic_compatibility(vp1, vp2, vk1, vk2):
    early_return = False
    early_result = None
    if vp1 and not vp2:
         early_return = True
         early_result = False
    if vk1 and not vk2:
        early_return = True
        early_result = False
    if vp2 and vk2:
        early_return = True
        early_result = True
    
    return early_return, early_result

def transform_key(d, func):
    return {func(k): v for k, v in d.items()}


kind_to_symbol = {PO: "PO", PK: "PK", VP: "VP", KO: "KO", VK: "VK"}


def param_kind_counter(sig):
    param_list = [param.kind for param in sig.parameters.values()]
    cc = Counter(param_list)
    res = transform_key(dict(cc), kind_to_symbol_func)
    return res


def kind_to_symbol_func(k):
    return kind_to_symbol[k]


def variadics_from_sig(sig):
    d = param_kind_counter(sig)
    vp = 'VP' in d
    vk = 'VK' in d

    return vp, vk

def is_valid_input_for_func(func, input):
    args, kwargs = input
    try:
        func(*args, **kwargs)
        return True
    except TypeError:
        return False

def remove_variadics_from_sig(sig):
    non_variadics_params = [param for param in sig.params if param.kind not in var_param_kinds]
    new_sig = Sig.from_params(non_variadics_params)

    return new_sig

def comp(sig1, sig2):
    vp1, vk1 = variadics_from_sig(sig1)
    vp2, vk2 = variadics_from_sig(sig2)
    
    early_return, early_result = variadic_compatibility(vp1, vp2, vk1, vk2)
    if early_return:
        return early_result
    
    new_sig1 = remove_variadics_from_sig(sig1)
    new_sig2 = remove_variadics_from_sig(sig2)
    func1 = sig_to_func(new_sig1)
    func2 = sig_to_func(new_sig2)
    inputs = sig_to_inputs(new_sig1)
    # assert valid for func1 !
    is_valid = all([is_valid_input_for_func(func2, input) for input in inputs])

    return is_valid

    

In [53]:
comp(sig1, sig1)
#param_kind_counter(sig1)

True

Signature: comp(sig1, sig2)
Docstring: <no docstring>
Source:   
def comp(sig1, sig2):
    vp1, vk1 = variadics_from_sig(sig1)
    vp2, vk2 = variadics_from_sig(sig2)

    early_return, early_result = variadic_compatibility(vp1, vp2, vk1, vk2)
    if early_return:
        return early_result

    new_sig1 = remove_variadics_from_sig(sig1)
    new_sig2 = remove_variadics_from_sig(sig2)
    func1 = sig_to_func(new_sig1)
    func2 = sig_to_func(new_sig2)
    inputs = sig_to_inputs(new_sig1)
    # assert valid for func1 !
    is_valid = all([is_valid_input_for_func(func2, input) for input in inputs])

    return is_valid
File:      ~/Desktop/dev/otosense/plunk/plunk/sb/sig_compatibility/sig_comp.py
Type:      function


In [1]:
from plunk.sb.sig_compatibility.sig_comp import comp

In [2]:
def myf(a00, a01, /, a12, a13, a14, a36):
    return (a00, a01, a12, a13, a14, a36)

In [10]:
from i2 import Sig
from i2.tests.util import generate_params

sig1 = Sig(generate_params([0, 0, 1, 1, 1, 3]))
sig2 = Sig(generate_params([0, 1, 1, 1,  3, 3]))

In [11]:
sig1, sig2

(<Sig (a00, a01, /, a12, a13, a14, *, a35)>,
 <Sig (a00, /, a11, a12, a13, *, a34, a35)>)

In [12]:
comp(sig1, sig2)

True